# AutoGen Function Calling

This tutorial will redo the day-3-function-calling-with-llamaIndex.ipynb tutorial using AutoGen.

The tutorial demonstrates how to use AutoGen to create agents that translate natural language queries into SQL. We'll focus on:

1. Designing effective agents for SQL translation
2. Setting up proper agent interactions
3. Using function calling to execute database operations
4. Creating natural conversations about data

First install the autogen-agentchat package.
```
%pip install autogen-agentchat~=0.2.0
```

## Basic Setup

First, set up the llm connection and sql connection.

In [12]:
import os
from dotenv import load_dotenv
import sqlite3

load_dotenv()

llm_list = [
    {
        "model": "gpt-4o",
        "api_type": "azure",
        "api_key": os.getenv("AZURE_OPENAI_KEY"),
        "api_version": os.getenv("OPENAI_API_VERSION"),
        "base_url": os.getenv("AZURE_OPENAI_ENDPOINT"),
    }
]

llm_config = {
    "temperature": 0.7,
    "config_list": llm_list,
    "timeout": 60,
}

# set up the sql connection
db_file = "sample.db"
db_conn = sqlite3.connect(db_file)

# Understanding AutoGen's Assist Agent and User Proxy

## Core Concepts

AutoGen provides a framework for building conversational AI agents that can interact with each other and with users. Two fundamental agent types in AutoGen are:

1. **AssistantAgent (Assist Agent)**
   - An AI-powered agent that can understand and respond to queries
   - Processes information and generates responses using an LLM
   - Can use tools/functions when configured (but cannot execute code)
   - Maintains conversation context
   - Typically plays the role of the "expert" or "assistant"

2. **UserProxyAgent (User Proxy)**
   - Acts as an intermediary between human users and assistant agents
   - Can execute code and functions
   - Manages user interactions
   - Can be configured to run autonomously or require human approval
   - Helps maintain conversation flow and state

## Implementation Example

Let's create a basic example showing how these agents interact:

```python
import autogen

# Configure the assistant
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config={
        "temperature": 0.7,
        "config_list": [{
            "model": "gpt-4",
            "api_key": "your_api_key",
            "base_url": "your api endpoint"
        }],
    },
    system_message="You are a helpful AI assistant that provides clear and concise answers."
)

# Configure the user proxy
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="TERMINATE",  # TERMINATE, NEVER, or ALWAYS
    max_consecutive_auto_reply=2,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "coding", # directory for code execution
        "use_docker": False, 
    },
)
```

### Key Features

#### AssistantAgent Features:
- **System Message**: Defines the agent's role and behavior
- **LLM Configuration**: Specifies the language model and parameters
- **Function Calling**: Can be configured with custom functions
- **Memory**: Maintains conversation history

#### UserProxyAgent Features:
- **Human Input Modes**:
  - `TERMINATE`: Asks for human input only when terminating
  - `NEVER`: Runs completely autonomously
  - `ALWAYS`: Always asks for human input
- **Code Execution**: Can execute code in a controlled environment
- **Auto-reply Settings**: Controls consecutive automatic responses
- **Termination Conditions**: Customizable conversation ending criteria


## Agent Design

In this section, we will implement the AutoGen function calling feature, which performs the same tasks as described in [function calling with LlamaIndex](day-3-function-calling-with-llamaIndex.ipynb).

We need three agents:
1. Planner: `AssistantAgent`. This agent plans the conversation and decides what to do next, and can summarize the code execution result to human. 
2. Data engineer: `AssistantAgent`. This agent can write and debug the sql query for a user to execute for a given task
3. Code executor: `UserProxyAgent`. This agent executes the code and returns the result to the planner.

First, we define all the tools.

In [25]:
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager
from typing import Annotated, List, Tuple

In [31]:
# Create planner agent
planner = AssistantAgent(
    name = 'planner',
    llm_config = llm_config,
    system_message = """
    You are a helpful AI assistant that plans database queries and analyzes results.
    Your job is to:
    1. Understand the user's question about the database
    2. Plan the necessary steps to answer the question
    3. Coordinate with the data engineer to get SQL queries
    4. Analyze the results and provide a clear, natural language answer to the user
    5. Remember context from previous questions to handle follow-up queries

    You don't write SQL directly - you delegate that to the data engineer. Focus on planning and explaining results.
    """
)

In [32]:
# Create teh data engineer agent
data_engineer = AssistantAgent(
    name = 'data_engineer',
    llm_config = llm_config,
    system_message ="""You are an expert SQL data engineer. Your job is to:
        1. Write correct SQL queries based on user requirements
        2. Use the available functions to explore the database schema
        3. Ensure all queries are safe and read-only (SELECT statements only)
        4. Validate queries before execution
        5. Debug and fix any SQL errors

        Only write SQL for SELECT operations - never modify the database.
        """     
)

In [33]:
# Create code executor agent
code_executor = UserProxyAgent(
    name='code_executor',
    human_input_mode='NEVER',
    code_execution_config={
        "work_dir": "sql_execution",
        "use_docker": False,
    },
    system_message="""You are a code executor agent that can run SQL queries and database functions.
    You have access to the tools defined to execute.

    You don't generate content - you only execute code and functions when requested.
    """
)
    

In AutoGen's architecture, a `UserProxyAgent` as user_proxy is necessary to initiate the conversation. It is the entry point for the conversation. It also controls the flow of the conversation including when to terminate it, when to ask for human input, and when to execute code.

In [34]:
# Create the user proxy for human interaction
user_proxy = UserProxyAgent(
    name="user",
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=3,
    code_execution_config={
        "work_dir": "sql_execution",
        "use_docker": False,
    }
)

In the following code, we define the tools that the agents can use.
There are two types of tools:
1. `register_for_llm`: Use when the response needs to be interpreted by the LLM. `data_engineer` have tools registered, it gets information about what tools exist and their parameters in its context window. Therefore `data_engineer` can generate a structured tool call request.
2. `register_for_execution`: The tool can be executed by the code executor agent.

In [35]:
@data_engineer.register_for_llm(description="List all tables in the database")
@code_executor.register_for_execution()
def list_table() -> Annotated[List[str], 'A list of tables in the database']:
    """
    List all tables in the database.
    """
    print(' -DB CALL: list_tables')
    cursor = db_conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    return [table[0] for table in tables]

@data_engineer.register_for_llm(description="Look up the table schema")
@code_executor.register_for_execution()
def describe_table(
    table_name: Annotated[str, 'The name of the table to describe']
    ) -> Annotated[List[Tuple[str, str]], 'A list of columns and their descriptions']:
    """
    Look up the table schema.

    Args:
        table_name (str): The name of the table to describe

    Returns: 
        List of columns where each entry is a tuple of (column_name, column_type)
    """
    print(' - DB CALL: describe_table')

    cursor = db_conn.cursor()
    cursor.execute(f"PRAGMA table_info({table_name});")
    return [(col[1], col[2]) for col in cursor.fetchall()]

@data_engineer.register_for_llm(description="Execute a SELECT statement")
@code_executor.register_for_execution()
def execute_query(sql: Annotated[str, 'The SQL query to execute']
    ) -> Annotated[List[List[str]], 'A list of rows, where each row is a list of column values']:
    """
    Execute a SELECT statement and return the results.

    Args:
        sql (str): The SQL query to execute

    Returns:
        List of lists where each inner list contains the row data.
    """
    print(' - DB CALL: execute_query')

    cursor = db_conn.cursor()
    cursor.execute(sql)
    return cursor.fetchall()

In [36]:
# check if those tools are registered correctly
data_engineer.llm_config['tools']

[{'type': 'function',
  'function': {'description': 'List all tables in the database',
   'name': 'list_table',
   'parameters': {'type': 'object', 'properties': {}, 'required': []}}},
 {'type': 'function',
  'function': {'description': 'Look up the table schema',
   'name': 'describe_table',
   'parameters': {'type': 'object',
    'properties': {'table_name': {'type': 'string',
      'description': 'The name of the table to describe'}},
    'required': ['table_name']}}},
 {'type': 'function',
  'function': {'description': 'Execute a SELECT statement',
   'name': 'execute_query',
   'parameters': {'type': 'object',
    'properties': {'sql': {'type': 'string',
      'description': 'The SQL query to execute'}},
    'required': ['sql']}}}]

**GroupChatManager**

- Manages the overall conversation flow between multiple agents
- Determines which agent should speak next based on the conversation context
- Handles message routing between agents
- Enforces conversation rules like maximum rounds
- Uses the LLM to select the next speaker based on the conversation context

In our example, we use default `speaker_selection_method` `"auto"` which selects the next speaker automatically by LLM. There are other options such as `"round_robin"` which selects the next speaker in a round-robin manner. `"random"` which selects the next speaker randomly. `"manual"` which allows the user to select the next speaker.

In [37]:
# set up group chat
group_chat = GroupChat(
    agents=[user_proxy, planner, data_engineer, code_executor],
    messages=[],
    max_round=50
)

manager = GroupChatManager(
    groupchat=group_chat,
    llm_config=llm_config
)

# Start the conversation
user_proxy.initiate_chat(
    manager,
    message="What is the cheapest product?"
)
    

user (to chat_manager):

What is the cheapest product?

--------------------------------------------------------------------------------



Next speaker: planner

planner (to chat_manager):

To determine the cheapest product in the database, we need to retrieve the product with the lowest price. Here is the plan:

1. **Identify the table**: Determine which table contains the product information, including prices.
2. **Retrieve the lowest price**: Run a query to find the product with the minimum price.

I'll ask the data engineer to write an SQL query to find the product with the lowest price. 

**Request to Data Engineer:**
Could you please provide an SQL query that retrieves the product with the lowest price, including the product name and price?

--------------------------------------------------------------------------------

Next speaker: data_engineer

data_engineer (to chat_manager):

***** Suggested tool call (call_ExLGSmYcwU1dbr0Q2d7wUQR4): list_table *****
Arguments: 
{}
***************************************************************************

-------------------------------------------------------------------

ChatResult(chat_id=None, chat_history=[{'content': 'What is the cheapest product?', 'role': 'assistant', 'name': 'user'}, {'content': "To determine the cheapest product in the database, we need to retrieve the product with the lowest price. Here is the plan:\n\n1. **Identify the table**: Determine which table contains the product information, including prices.\n2. **Retrieve the lowest price**: Run a query to find the product with the minimum price.\n\nI'll ask the data engineer to write an SQL query to find the product with the lowest price. \n\n**Request to Data Engineer:**\nCould you please provide an SQL query that retrieves the product with the lowest price, including the product name and price?", 'name': 'planner', 'role': 'user'}, {'content': '', 'tool_calls': [{'id': 'call_ExLGSmYcwU1dbr0Q2d7wUQR4', 'function': {'arguments': '{}', 'name': 'list_table'}, 'type': 'function'}], 'name': 'data_engineer', 'role': 'assistant'}, {'content': '["products", "sqlite_sequence", "staff", "or